- Loading the Dataset

In [ ]:
import pandas as pd
import pickle

EXCEL_FILE_PATH = "../Data/MI 2 Rows of Couties North of IN-OH.xlsx"

# Load the Excel file
df = pd.read_excel()

# Data cleaning
df.dropna(subset=['MAIL_ADDR','MAIL_CITY','MAIL_STATE','MAIL_ZIP','OWNER_NAME_1'], inplace=True)
df = df.drop_duplicates(['MAIL_ADDR','MAIL_CITY','MAIL_STATE','MAIL_ZIP'])

In [ ]:


# Load the trained model
with open("logreg_classifier.pickle", "rb") as model_file:
    model = pickle.load(model_file)

# Load the TF-IDF vectorizer (you should also save the vectorizer when you save the model)
with open("logreg_vectorizer.pickle", "rb") as vec_file:
    vectorizer = pickle.load(vec_file)

# Ensure the 'OWNER_NAME_1' column exists
if 'OWNER_NAME_1' in df.columns:
    # Vectorize the 'Owner' column
    X_test = vectorizer.transform(df['OWNER_NAME_1'])
    
    # Predict using the loaded model
    df['Prediction'] = model.predict(X_test)
    
    # Optionally, save the DataFrame with predictions
    # df.to_excel("AR Top 10 Zips July 31 2024 - 3-9.99 ac Testing_Predictions.xlsx", index=False)
else:
    print("The 'Owner' column is missing from the Excel file.")


In [10]:
df = df[['OWNER_NAME_1', 'Prediction']]
df.to_excel(f"{EXCEL_FILE_PATH} SCRUBBED", index=False)

In [4]:
df = pd.read_excel(f"{EXCEL_FILE_PATH} SCRUBBED")
filtered_names = pd.read_excel("AR Top 10 Zips July 31 2024 - 3-9.99 ac Tested.xlsx")
human_names = filtered_names[filtered_names['Prediction'] == 'Human Name']['OWNER_NAME_1'].tolist()
df[df['OWNER_NAME_1'].isin(human_names)]['OWNER_NAME_1']


0          KOLLER, GREGORY
1             LOONEY, BILL
2       CHRISMAN, ANGELA D
3           SIKES, CHRISTY
4                  MOORE O
               ...        
3972         VANESS, ROBIN
3973        GRAY, DONALD D
3974        SABO, AMANDA L
3975    DAVENPORT, JAMES M
3976       MYERS, GEORGE S
Name: OWNER_NAME_1, Length: 3414, dtype: object

In [ ]:
# List of keywords indicating company names
Suspected_Words = [
    'OF', 'SERVICES', 'COUNTY', 'DISTRIBUTOR', 'PRODUCTS', 'COUNTRY CLUB', 'PLLC', 'PRIVATE',
    'COMPANY', 'INC', 'RETIREMENT', 'DEVELOPMENT', 'HOA', 'AUTHORITY', 'CONF', 'CONFERENCE',
    'CONSTRUCTION', 'AFFORDABLE', 'HOUSING', 'MID COAST', 'ESTATE', 'REHABILITATION', 'GARDENS',
    'WELLNESS', 'LLC', 'STATE', 'LAND', 'MEMBERSHIP', 'COOPERATIVE', 'CORP', 'CORPORATION',
    'INDEPENDENT', 'CHURCH', 'ENTERPRISES', 'ACCOUNTING', 'INVESTMENTS', 'OWNERS', 'AIRPORT',
    'MAINTENANCE', 'ASSOCIATION', 'REALTY', 'FOUNDATION', 'CONSULTANTS', 'ASSOCIATES',
    'CORPORATE', 'DISTRICT', 'LTD', 'LIMITED', 'INCORPORATED', 'PROPERTIES', 'INVESTMENT',
    'NORTHEAST', 'PLUMBING', 'HEATING'
]

# Pattern to detect suspected company names
pattern = r'\b(' + '|'.join(Suspected_Words) + r')\b'

# Filter out suspected company names
df = df[~df['OWNER_NAME_1'].str.contains(pattern, case=False, na=False)]